# To create a calculator tool

In [ ]:
from pydantic import BaseModel, Field

class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class subtract(BaseModel):
    """Subtract two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class divide(BaseModel):
    """Divide two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

## Custom tool for addition, subtraction, division and multiplication

### Usign structured tool

In [38]:
from langchain_core.tools import tool

def add_numbers(a: int, b: int) -> int:
    """
    Adds two numbers together.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of the two numbers.
    """
    return a + b

def subtract_numbers(a: int, b: int) -> int:
    """
    Subtracts the second number from the first.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The result of subtracting b from a.
    """
    return a - b

def multiply_numbers(a: int, b: int) -> int:
    """
    Multiplies two numbers together.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The product of the two numbers.
    """
    return a * b

def divide_numbers(a: int, b: int) -> float:
    """
    Divides the first number by the second.

    Args:
        a (int): The numerator.
        b (int): The denominator.

    Returns:
        float: The result of dividing a by b.
    
    Raises:
        ValueError: If b is zero, as division by zero is not allowed.
    """
    if b == 0:
        raise ValueError("Cannot divide by zero.")
    return a / b

In [42]:
from langchain_core.tools import tool
from langchain_core.tools import StructuredTool

add_tool = StructuredTool.from_function(
    func = add_numbers,
    name = "add_numbers",
    description = "Adds two numbers together.",
    args_schema = add
)

subtract_tool = StructuredTool.from_function(
    func = subtract_numbers,
    name = "subtract_numbers",
    description = "Subtracts the second number from the first.",
    args_schema = subtract
)

multiply_tool = StructuredTool.from_function(
    func = multiply_numbers,
    name = "multiply_numbers",
    description = "Multiplies two numbers together.",
    args_schema = multiply
)

divide_tool = StructuredTool.from_function(
    func = divide_numbers,
    name = "divide_numbers",
    description = "Divides the first number by the second.",
    args_schema = divide
)

In [47]:
def math1():
    a = input("Enter first number: ")
    b = input("Enter second number: ")

    subtract_result = subtract_tool.invoke({"a": int(a), "b": int(b)})
    add_result = add_tool.invoke({"a": int(a), "b": int(b)})
    multiply_result = multiply_tool.invoke({"a": int(a), "b": int(b)})
    divide_result = divide_tool.invoke({"a": int(a), "b": int(b)})

    print(f"Subtraction Result: {subtract_result}")
    print(f"Addition Result: {add_result}")
    print(f"Multiplication Result: {multiply_result}")
    print(f"Division Result: {divide_result}")


In [48]:
math1()

Subtraction Result: 0
Addition Result: 10
Multiplication Result: 25
Division Result: 1.0


### Using BaseTool

In [159]:
from langchain.tools import BaseTool
from typing import Type

class add_tool(BaseTool):
    name: str = "add"
    description: str = "Adds two integers."
    args_schema: Type[add] = add

    def _run(self, a: int, b: int) -> int:
        return a+b
    
class subtract_tool(BaseTool):
    name: str = "subtract"
    description: str = "Subtracts two integers."
    args_schema: Type[subtract] = subtract

    def _run(self, a: int, b: int) -> int:
        return a-b

class multiply_tool(BaseTool):
    name: str = "multiply"
    description: str = "Multiplies two integers."
    args_schema: Type[multiply] = multiply

    def _run(self, a: int, b: int) -> int:
        return a*b
    
class divide_tool(BaseTool):
    name: str = "divide"
    description: str = "Divides two integers."
    args_schema: Type[divide] = divide

    def _run(self, a: int, b: int) -> float:
        if b == 0:
            raise ValueError("Cannot divide by zero.")
        return a/b
    


In [160]:
addition = add_tool()
subtraction = subtract_tool()
multiplication = multiply_tool()
division = divide_tool()

In [161]:
addition_result = addition.invoke({"a": 5, "b": 3})
subtraction_result = subtraction.invoke({"a": 5, "b": 3})
multiplication_result = multiplication.invoke({"a": 5, "b": 3})
division_result = division.invoke({"a": 5, "b": 3})


print(f"Addition Result: {addition_result}")
print(f"Subtraction Result: {subtraction_result}")
print(f"Multiplication Result: {multiplication_result}")
print(f"Division Result: {division_result}")

Addition Result: 8
Subtraction Result: 2
Multiplication Result: 15
Division Result: 1.6666666666666667


### Tool Calling in chatbot

In [126]:
import os

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"


#### Using base tool to bind  into the chatbot

In [127]:
addition.name, subtraction.name, multiplication.name, division.name

('add', 'subtract', 'multiply', 'divide')

In [128]:
print("add: ", addition.args) 
print("subtract: ", subtraction.args) 
print("multiplication: ", multiplication.args)
print("Division: ", division.args)

add:  {'a': {'description': 'First integer', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second integer', 'title': 'B', 'type': 'integer'}}
subtract:  {'a': {'description': 'First integer', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second integer', 'title': 'B', 'type': 'integer'}}
multiplication:  {'a': {'description': 'First integer', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second integer', 'title': 'B', 'type': 'integer'}}
Division:  {'a': {'description': 'First integer', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second integer', 'title': 'B', 'type': 'integer'}}


In [185]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0.7,
)


In [186]:
llm_with_tools = llm.bind_tools([addition, subtraction, multiplication, division])

#### Tool Calling

In [188]:
from langchain_core.messages import HumanMessage

query = HumanMessage('What is the sum of 5 and 3?')
messages = [query]
result = llm_with_tools.invoke("What is the sum of 5 and 3?")
messages.append(result)
messages

if result.tool_calls[0]['name'] == "add":
    tool_result = addition.invoke(result.tool_calls[0])
if result.tool_calls[0]['name'] == "subtract":
    tool_result = subtraction.invoke(result.tool_calls[0])
if result.tool_calls[0]['name'] == "multiply":
    tool_result = multiplication.invoke(result.tool_calls[0])
if result.tool_calls[0]['name'] == "divide":
    tool_result = division.invoke(result.tool_calls[0])

messages.append(tool_result)

llm_with_tools.invoke(messages).content

'8'

In [189]:
result.tool_calls[0]

{'name': 'add',
 'args': {'a': 5, 'b': 3},
 'id': 'q6j3bywy0',
 'type': 'tool_call'}

In [191]:
result.tool_calls[0]['args']

{'a': 5, 'b': 3}

#### Tool Execution

In [174]:
addition.invoke(result.tool_calls[0])

ToolMessage(content='8', name='add', tool_call_id='eemesa4ab')

Addition:  content='8' name='add' tool_call_id='eemesa4ab'


# Multi-Tool Agent Assistant (Web Search + Calculator + Weather)

In [214]:
from dotenv import load_dotenv
import os 
load_dotenv()

SERP_API_KEY = os.getenv("SERPAPI_API_KEY")

In [237]:
pip uninstall serpapi google-search-results -y


Found existing installation: google_search_results 2.4.2
Uninstalling google_search_results-2.4.2:
  Successfully uninstalled google_search_results-2.4.2
Note: you may need to restart the kernel to use updated packages.


In [238]:
pip install google-search-results

  Using cached google_search_results-2.4.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [244]:
pip install --upgrade langchain serpapi


  Using cached serpapi-0.1.5-py2.py3-none-any.whl.metadata (10 kB)
Using cached serpapi-0.1.5-py2.py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.


### Creating tools for search and calculation

In [9]:
from langchain.utilities.serpapi import SerpAPIWrapper

In [ ]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
# from langchain.utilities.serpapi import SerpAPIWrapper # We will not use this directly
import serpapi # Import the serpapi library directly
import os # To access environment variables for API key

# Creating a pydantic base model for the input schema
class CelsiusToFahrenheitInput(BaseModel):
    """Input schema for Celsius to Fahrenheit conversion."""
    celsius: float = Field(..., description="Temperature in Celsius")

class WeatherInput(BaseModel):
    """Input schema for weather tool."""
    location: str = Field(..., description="Location to fetch the weather for")

# Creating a tool class for Celsius to Fahrenheit conversion
class CelsiusToFahrenheitTool(BaseTool):
    name: str = "celsius_to_fahrenheit"
    description: str = "Converts Celsius to Fahrenheit."
    args_schema: Type[BaseModel] = CelsiusToFahrenheitInput

    def _run(self, celsius: float) -> float:
        celsius = float(celsius)
        return (celsius * 9/5) + 32

class WeatherTool(BaseTool):
    name: str = "weather"
    description: str = "Fetches the current weather for a given location using SerpAPI."
    args_schema: Type[BaseModel] = WeatherInput

    def _run(self, location: str) -> str:
        try:
            api_key = os.getenv("SERPAPI_API_KEY")
            if not api_key:
                return "Error: SERPAPI_API_KEY not found in environment variables."

            params = {
                "q": f"temperature in {location}",
                "engine": "google",
                "api_key": api_key
            }
            search_results = serpapi.search(params)
            
            # Structured results are in "answer_box"
            if "answer_box" in search_results and "temperature" in search_results["answer_box"]:
                return f"The temperature in {location} is {search_results['answer_box']['temperature']}."
            else:
                return f"Could not find temperature information for {location}."

        except Exception as e:
            return f"Error occurred: {str(e)}"

    def _arun(self, location: str):
        raise NotImplementedError("Async version not implemented.")



In [11]:
from langchain.agents import initialize_agent, AgentType
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0.7,
)

# Instantiate Tools
celsius_tool = CelsiusToFahrenheitTool()
weather_tool = WeatherTool()

# Initialize the Agent with tools
agent = initialize_agent(
    tools=[celsius_tool, weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [12]:
response = agent.run("What’s the weather in New York and what’s 5°C in Fahrenheit?")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to get the weather in New York and convert 5°C to Fahrenheit.
Action: weather
Action Input: New York
Observation: The temperature in New York is 77.
Thought:Thought: Now I need to convert 5°C to Fahrenheit. 
Action: celsius_to_fahrenheit
Action Input: 5
Observation: 41.0
Thought:Thought: I now know the final answer
Final Answer: The weather in New York is 77 degrees and 5°C is 41 degrees Fahrenheit.  


> Finished chain.
The weather in New York is 77 degrees and 5°C is 41 degrees Fahrenheit.
